# 🌿 My Garden Care - Cloud Computing Project

**Team: Shark Team**

---

## Instructions:
1. **Run Cell 1** to install all dependencies
2. **Upload** `serviceAccountKey.json` to the Colab file browser
3. **Run Cell 2** to set your Google API key
4. **Run all %%writefile cells** to create project files
5. **Create `articles_data/` folder** and upload knowledge base files
6. **Run the final cell** to launch the application

## 📦 Cell 1: Install Dependencies

In [ ]:
!pip install -q firebase-admin nltk google-cloud-firestore google-auth numpy gradio matplotlib requests python-docx scikit-learn sentence-transformers chromadb google-generativeai python-dotenv

## 🔑 Cell 2: Security Setup (Colab Secrets)

**Before running this cell, add these secrets:**

1. Click the **🔑 Key icon** (Secrets) in the left sidebar
2. Add **two secrets**:

| Secret Name | Value |
|-------------|-------|
| `GOOGLE_API_KEY` | Your Gemini API key |
| `FIREBASE_JSON` | Copy & paste the **entire content** of your `serviceAccountKey.json` file |

3. Toggle the switch to enable notebook access for both secrets

In [ ]:
import os
from google.colab import userdata

# 1. Setup Gemini API Key
try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print('✅ Gemini API Key loaded from Secrets.')
except Exception:
    print('❌ Error: GOOGLE_API_KEY secret is missing.')
    print('   Add it in the 🔑 Secrets panel on the left sidebar.')

# 2. Setup Firebase Credentials
try:
    # User pastes the content of serviceAccountKey.json into this secret
    firebase_json_content = userdata.get('FIREBASE_JSON')
    
    # Write it to a file so the app can use it normally
    with open('serviceAccountKey.json', 'w') as f:
        f.write(firebase_json_content)
    
    os.environ['FIREBASE_CREDENTIALS_PATH'] = '/content/serviceAccountKey.json'
    print('✅ Firebase credentials created from Secrets.')
except Exception:
    print('❌ Error: FIREBASE_JSON secret is missing.')
    print('   Copy the entire content of your serviceAccountKey.json file')
    print('   and paste it into a secret named FIREBASE_JSON.')

## 📁 Cell 3: Create Project Structure

In [ ]:
import os
os.makedirs('ui', exist_ok=True)
os.makedirs('articles_data', exist_ok=True)
print('✅ Directories created: ui/, articles_data/')

### Cell 4: config.py

In [ ]:
%%writefile config.py
import os
import sys
import firebase_admin
from firebase_admin import credentials, firestore
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# ==========================================
# PART 1: ENVIRONMENT & PATH CONFIGURATION
# ==========================================

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Specific path for Colab environment
    PROJECT_ROOT = "/content/Cloud_Computing_Project_Shark_Team"
else:
    # Local path (dynamic based on file location)
    PROJECT_ROOT = os.path.dirname(os.path.abspath(__file__))

# Check for environment variable first, then fall back to default
KEY_FILENAME = "serviceAccountKey.json"
DEFAULT_CRED_PATH = os.path.join(PROJECT_ROOT, KEY_FILENAME)

# Priority: ENV variable > Default path > Current directory
FIREBASE_CRED_PATH = os.getenv("FIREBASE_CREDENTIALS_PATH", DEFAULT_CRED_PATH)

# Validate key existence
if not os.path.exists(FIREBASE_CRED_PATH):
    # Fallback: check current directory if configured path fails
    if os.path.exists(KEY_FILENAME):
        FIREBASE_CRED_PATH = KEY_FILENAME
    else:
        print(f"Warning: Firebase key not found at {FIREBASE_CRED_PATH}")

# ==========================================
# PART 2: DATABASE INITIALIZATION (SINGLETON)
# ==========================================

import json

# Global variable to hold the DB instance
_DB_CLIENT = None

def get_db():
    """Returns singleton Firestore client. Initializes Firebase if needed."""
    global _DB_CLIENT
    
    # Return existing instance if available
    if _DB_CLIENT is not None:
        return _DB_CLIENT

    # Check if Firebase is already initialized internally
    if not firebase_admin._apps:
        try:
            # 1. Read the JSON key to get project_id (for storageBucket)
            bucket_name = None
            if os.path.exists(FIREBASE_CRED_PATH):
                with open(FIREBASE_CRED_PATH, "r") as f:
                    key_data = json.load(f)
                    pid = key_data.get("project_id")
                    if pid:
                        # Newer Firebase projects use .firebasestorage.app
                        bucket_name = f"{pid}.firebasestorage.app"
            
            # 2. Initialize App with Storage Bucket
            cred = credentials.Certificate(FIREBASE_CRED_PATH)
            options = {"storageBucket": bucket_name} if bucket_name else None
            
            firebase_admin.initialize_app(cred, options=options)
            
            print(f"[System] Firebase initialized using {FIREBASE_CRED_PATH}")
            if bucket_name:
                print(f"[System] Cloud Storage Bucket: {bucket_name}")
                
        except Exception as e:
            print(f"[Critical Error] Failed to init Firebase: {e}")
            raise e
    
    _DB_CLIENT = firestore.client()
    return _DB_CLIENT

### Cell 5: gamification_rules.py

In [ ]:
%%writefile gamification_rules.py
"""
GAMIFICATION RULES MODULE
-------------------------
This file defines the points system and weekly challenges.
It serves as the "Rule Book" for the Main Application.

KEY RULES:
1. Points are PER PLANT. If a user waters 5 plants, they get points 5 times.
2. Weekly Challenge Counters must reset every week (Sunday).
3. Total Score NEVER resets.
"""

import datetime

# ==========================================
# 1. POINTS SYSTEM (The "Price List")
# ==========================================
# Simple Logic: Action Performed -> Points Awarded immediately.

ACTIONS = {
    "WATER_PLANT": {
        "points": 10,
        "description": "Watering a single plant"
    },
    "FERTILIZE_PLANT": {
        "points": 10,
        "description": "Fertilizing a single plant"
    },
    "UPLOAD_PHOTO": {
        "points": 20,
        "description": "Scanning a plant with AI"
    },
    "EARLY_DETECTION": {
        "points": 40,
        "description": "System detected an issue early"
    },
    "PREVENTIVE_ACTION": {
        "points": 25,
        "description": "User completed a maintenance task"
    },
    "USE_SEARCH": {
        "points": 0,
        "description": "Using the RAG Knowledge Base"
    },
    "ADD_PLANT": {
        "points": 15,
        "description": "Adding a new plant to the garden"
    }
}

# ==========================================
# 2. WEEKLY CHALLENGES
# ==========================================
# Cycle through these challenges (one per week).

WEEKLY_CHALLENGES = [
    {
        "id": 1,
        "title": "Photo Marathon",
        "description": "Scan 3 different plants to update their status.",
        "action_type": "UPLOAD_PHOTO",
        "target": 3, 
        "reward_points": 150
    },
    {
        "id": 2,
        "title": "Garden Expansion",
        "description": "Add a new plant to your garden collection.",
        "action_type": "ADD_PLANT",
        "target": 1,
        "reward_points": 100
    },
    {
        "id": 3,
        "title": "The Scholar",
        "description": "Use the Smart Search (RAG) to ask a question about plants.",
        "action_type": "USE_SEARCH",
        "target": 1, 
        "reward_points": 50
    }
]

# ==========================================
# 3. HELPER FUNCTIONS
# ==========================================

# Global variable to store the forced challenge ID (None = Auto/Calendar Mode)
_FORCED_CHALLENGE_ID = None

def set_challenge_mode(mode_id):
    """
    Sets the challenge mode.
    None = Use Real Calendar.
    1, 2, 3 = Force specific challenge ID.
    """
    global _FORCED_CHALLENGE_ID
    _FORCED_CHALLENGE_ID = mode_id

def get_points_for_action(action_key):
    """
    Returns the points for a specific action.
    """
    if action_key in ACTIONS:
        return ACTIONS[action_key]["points"]
    return 0

def get_current_weekly_challenge():
    # 1. Check if a specific challenge is forced (Manual Mode)
    if _FORCED_CHALLENGE_ID is not None:
        # Find the challenge with the specific ID
        for challenge in WEEKLY_CHALLENGES:
            if challenge['id'] == _FORCED_CHALLENGE_ID:
                return challenge
    
    # 2. If no force (Auto Mode), use the Real Calendar logic
    # Calculate week number (1-52)
    current_week = datetime.date.today().isocalendar()[1]
    
    # Cycle through challenges
    challenge_index = current_week % len(WEEKLY_CHALLENGES)
    return WEEKLY_CHALLENGES[challenge_index]

def get_user_rank(current_score):
    """
    Simple rank calculation based on total score.
    """
    if current_score < 100:
        return "Beginner"
    elif current_score < 500:
        return "Advanced"
    elif current_score < 1000:
        return "Pro"
    else:
        return "Master"

# ==========================================
# TEST BLOCK
# ==========================================
if __name__ == "__main__":
    print("--- GAMIFICATION RULES CHECK ---")
    print(f"Adding Plant Points: {get_points_for_action('ADD_PLANT')}")
    print(f"Current Challenge: {get_current_weekly_challenge()['title']}")
    print("Rules are valid.")

### Cell 6: auth_service.py

In [ ]:
%%writefile auth_service.py
import hashlib
import datetime
from firebase_admin import firestore  
from config import get_db
import gamification_rules
from plants_manager import clear_plants_cache

db = get_db()

# ==========================================
# HELPER FUNCTION: SECURITY
# ==========================================

def _hash_password(password):
    """SHA-256 hash of password."""
    password_bytes = password.encode('utf-8')
    hash_object = hashlib.sha256(password_bytes)
    return hash_object.hexdigest()

# ==========================================
# AUTHENTICATION FUNCTIONS
# ==========================================

def logout_user():
    """Clears cached user data on logout."""
    clear_plants_cache()
    return True

def register_user(username, display_name, password, email):
    try:
        # Validation: Ensure fields are not empty 
        if not username or not password or not email or not display_name:
            return False, "Error: All fields are required."

        # Check for spaces in usrename
        if " " in username:
            return False, "Error: Username cannot contain spaces (use 'carmel12' not 'carmel 12')."

        if len(password) < 6:
            return False, "Error: Password must be at least 6 characters long."

        # Validate email format
        import re
        email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(email_pattern, email):
            return False, "Error: Please enter a valid email address (e.g., user@example.com)."

        # 1. Check if user already exists
        users_ref = db.collection('users').document(username)
        doc = users_ref.get()
        
        if doc.exists:
            return False, "Error: Username already exists."

        hashed_password = _hash_password(password)
        
        # 2. Create user data object
        new_user = {
            'username': username,      # The technical ID (e.g. 'zohar_l')
            'display_name': display_name, # The real name (e.g. 'Zohar Levy')
            'password': hashed_password,
            'email': email,
            'score': 0,
            'tasks_completed': 0,
            'created_at': firestore.SERVER_TIMESTAMP
        }
        
        # 3. Save to Firestore
        users_ref.set(new_user)
        return True, "Success: User registered successfully."
        
    except Exception as e:
        return False, f"System Error: {str(e)}"

def login_user(username, password):
    """
    Authenticates a user by comparing password hashes.
    """
    try:
        # Validation: Ensure fields are not empty 
        if not username or not password:
            return False, "Error: Username and Password fields cannot be empty."

        # Step 1: Fetch user from Firestore 
        users_ref = db.collection('users').document(username)
        doc = users_ref.get()

        # Step 2: Check if user exists 
        if not doc.exists:
            return False, "Error: User not found. Please register first."

        # Step 3: Verify Password 
        user_data = doc.to_dict()
        stored_hash = user_data.get('password')
        
        # Security: Hash the input password to compare with the stored hash
        input_hash = _hash_password(password)
        
        if stored_hash == input_hash:
            return True, user_data  # Login Success
        else:
            return False, "Error: Incorrect password."
            
    except Exception as e:
        return False, f"System Error: {str(e)}"

# ==========================================
# GAMIFICATION FUNCTIONS
# ==========================================

def update_user_score(username, points):
    try:
        user_ref = db.collection('users').document(username)
        doc = user_ref.get()
        if doc.exists:
            user_ref.update({
                'score': firestore.Increment(points),
                'tasks_completed': firestore.Increment(1)
            })
            updated_doc = user_ref.get()
            return updated_doc.to_dict().get('score')
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_leaderboard():
    try:
        users_ref = db.collection('users')
        query = users_ref.order_by('score', direction=firestore.Query.DESCENDING).limit(5)
        results = query.stream()
        
        leaderboard_data = []
        for doc in results:
            data = doc.to_dict()
            leaderboard_data.append({
                'username': data.get('username'),
                'score': data.get('score')
            })
        return leaderboard_data
    except Exception as e:
        print(f"Error: {e}")
        return []

def update_weekly_challenge_progress(username, action_type):
    """
    Updates the user's progress for the current weekly challenge.
    Handles weekly resets logic automatically.
    """
    try:
        user_ref = db.collection('users').document(username)
        doc = user_ref.get()
        
        if not doc.exists:
            return None

        user_data = doc.to_dict()
        
        # 1. Identify the current active challenge
        current_challenge = gamification_rules.get_current_weekly_challenge()
        challenge_id = str(current_challenge['id'])
        target_action = current_challenge['action_type']
        
        # 2. Check if the action is relevant to the current challenge
        if action_type != target_action:
            return {
                "relevant": False,
                "msg": "Action does not match weekly challenge."
            }

        # 3. Get User's challenge state (or initialize it)
        challenge_state = user_data.get('challenge_state', {})
        
        # Check if we need to reset (if the stored challenge ID is old)
        stored_id = challenge_state.get('challenge_id')
        
        if stored_id != challenge_id:
            # New week detected! Reset counters.
            challenge_state = {
                'challenge_id': challenge_id,
                'progress': 0,
                'is_completed': False,
                'last_updated': firestore.SERVER_TIMESTAMP
            }

        # If already completed, return full data structure to prevent KeyErrors
        if challenge_state.get('is_completed'):
             return {
                "relevant": True,
                "completed": True,
                "progress": challenge_state.get('progress', current_challenge['target']), 
                "target": current_challenge['target'],
                "bonus_awarded": 0,
                "msg": "Challenge already completed for this week."
            }

        # 5. Update Progress
        new_progress = challenge_state['progress'] + 1
        challenge_target = current_challenge['target']
        bonus_points = 0
        is_finished = False

        if new_progress >= challenge_target:
            is_finished = True
            bonus_points = current_challenge['reward_points']
            challenge_state['is_completed'] = True
            
            # Grant the Bonus Points!
            update_user_score(username, bonus_points)

        challenge_state['progress'] = new_progress
        
        # 6. Save back to Firestore
        user_ref.update({
            'challenge_state': challenge_state
        })

        return {
            "relevant": True,
            "progress": new_progress,
            "target": challenge_target,
            "completed": is_finished,
            "bonus_awarded": bonus_points
        }

    except Exception as e:
        print(f"Error in challenge update: {e}")
        return None
    
    
def get_user_details(username):
    """
    Fetches user data without requiring a password.
    Used for refreshing the dashboard after an action.
    """
    try:
        users_ref = db.collection('users').document(username)
        doc = users_ref.get()
        if doc.exists:
            return doc.to_dict()
        return None
    except Exception as e:
        print(f"Error fetching user details: {e}")
        return None

### Cell 7: plants_manager.py

In [ ]:
%%writefile plants_manager.py
"""
plants_manager.py

Plants belong to a specific user.

Firestore data model:
  users/{username}/plants/{plant_id}

Each plant document can store:
- name (display name)
- species (optional)
- image_url (recommended when using cloud storage)
- image_path (optional local fallback)
"""

from __future__ import annotations

from datetime import datetime, timezone
import uuid
from typing import Any
import os
import re
import time

# --- New Imports for AI ---
import google.generativeai as genai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

from config import get_db


# ==========================================
# CACHING LAYER (TTL-based)
# ==========================================

_plants_cache: dict[str, tuple[list, float]] = {}  # {username: (plants_list, timestamp)}
_CACHE_TTL_SECONDS = 60  # Cache expires after 60 seconds


def clear_plants_cache(username: str = None):
    """Clears plant list cache. If username given, only that user's cache."""
    global _plants_cache
    if username:
        _plants_cache.pop(username, None)
    else:
        _plants_cache.clear()


def _utc_now_iso() -> str:
    """Return current UTC time as ISO string."""
    return datetime.now(timezone.utc).isoformat()


def _clean(s: Any) -> str:
    """Convert input to a trimmed string safely."""
    return str(s).strip() if s is not None else ""

def get_optimal_soil(species_name: str) -> int:
    """Uses Gemini AI to get optimal soil moisture for a plant. Returns 30 on failure."""
    api_key = os.getenv("GOOGLE_API_KEY")
    
    # Safety check: If no API key is found, return default immediately
    if not api_key:
        print("[AI Warning] No GOOGLE_API_KEY found in .env. Using default 30%.")
        return 30

   # List of models to try in order (Fallback mechanism)
    models_to_try = [
        'gemini-2.0-flash',       # First priority
        'gemini-2.5-flash',       # Second priority
        'gemini-flash-latest',    # Fallback generic name
        'models/gemini-2.0-flash' # Explicit path just in case
    ]
    
    genai.configure(api_key=api_key)

    # Clean the input
    species_name = species_name.strip()

    prompt = f"""
    You are an expert agronomist. 
    I am growing a plant of type: "{species_name}".
    What is the critical minimum soil moisture percentage (0-100%) this plant needs to survive before it starts wilting?
    Note: I am measuring soil moisture, NOT air humidity.
    Return ONLY the number (integer). No text.
    Example response: 30
    """
    # This loop was causing the indentation error - now fixed:
    for model_name in models_to_try:
        try:
            print(f"[AI Agent] Connecting to model: {model_name} for '{species_name}'...")
            model = genai.GenerativeModel(model_name)
            
            response = model.generate_content(prompt)
            text = response.text.strip()
            
            numbers = re.findall(r'\d+', text)
            if numbers:
                val = int(numbers[0])
                if 5 <= val <= 90:
                    return val
            
        except Exception as e:
            print(f"[AI Log] Model {model_name} failed: {e}")
            continue

    print("[AI Error] All models failed. Using fallback 30%.")
    return 30

def add_plant(
    username: str,
    name: str,
    species: str = "",
    image_url: str = "",
    image_path: str = "",
) -> tuple[bool, str]:
    """Creates a plant document with AI-powered soil threshold detection."""
    username = _clean(username)
    name = _clean(name)
    species = _clean(species)
    image_url = _clean(image_url)
    image_path = _clean(image_path)

    if not username:
        return False, "Missing username (please login)."
    if not name:
        return False, "Plant name is required."

    # --- AI Integration Start ---
    # Determine minimum humidity based on species
    optimal_min = 30 # Default value

    # Logic: If species is provided, use it. Otherwise, try to infer from the name.
    ai_search_term = species if species else name

    if ai_search_term:
        # Call the helper function to get the SOIL threshold
        optimal_min = get_optimal_soil(ai_search_term)
        print(f"[Success] AI set soil threshold for '{name}' to {optimal_min}% (Term: '{ai_search_term}')")
    # --- AI Integration End ---

    plant_id = uuid.uuid4().hex[:8]
    doc = {
        "plant_id": plant_id,
        "name": name,
        "species": species,
        "min_soil": optimal_min, # <--- Storing the AI result in DB
        "image_url": image_url,
        "image_path": image_path,
        "created_at": _utc_now_iso(),
    }

    db = get_db()
    try:
        db.collection("users").document(username).collection("plants").document(plant_id).set(doc)
        return True, plant_id
    except Exception as e:
        return False, f"Failed to add plant: {e}"


import json
import os
import google.generativeai as genai

import json
import os
import re
import google.generativeai as genai

def get_vacation_advice_ai(plant_name, current_soil, min_threshold, current_temp, days_away):
    api_key = os.getenv("GOOGLE_API_KEY")
    if not api_key:
        print("[System] Missing API Key.")
        return None

    genai.configure(api_key=api_key)

    # List of models to try in order (Fallback mechanism), same as in get_optimal_soil
    models_to_try = [
        'gemini-2.0-flash',       # First priority
        'gemini-2.5-flash',       # Second priority
        'gemini-flash-latest',    # Fallback generic name
        'models/gemini-2.0-flash' # Explicit path just in case
    ]

    # Construct the prompt
    prompt = f"""
    You are an expert botanist.
    I am going on vacation for {days_away} days.
    
    Plant Details:
    - Type: {plant_name}
    - Current Soil Moisture: {current_soil}%
    - Minimum Survival Threshold: {min_threshold}%
    
    Environment Conditions (Real-time Sensor):
    - Indoor Temperature: {current_temp}°C
    
    Task:
    1. Analyze if the temperature indicates fast evaporation (Summer/Hot) or slow (Winter/Cold).
    2. Estimate the daily soil moisture loss % for this specific plant at this temperature.
    3. Determine if the plant will survive without intervention.
    4. Recommend action: "Water heavily now" OR "Must install automatic irrigation system".
    
    Return ONLY a valid JSON object in this format:
    {{
        "status": "SAFE" or "NEEDS WATER" or "CRITICAL",
        "message": "Short explanation mentioning temp effect (e.g., 'High heat (30C) increases drying rate')",
        "recommendation": "The specific action to take"
    }}
    """

    # Retry Loop: Try each model until one succeeds
    for model_name in models_to_try:
        try:
            print(f"[AI Agent] Connecting to model: {model_name} for vacation advice...")
            model = genai.GenerativeModel(model_name)
            
            response = model.generate_content(prompt)
            text = response.text.strip()
            
            # Clean up Markdown formatting if present (```json ... ```)
            if text.startswith("```json"):
                text = text[7:-3].strip()
            elif text.startswith("```"):
                text = text[3:-3].strip()
            
            # Try to parse JSON
            result = json.loads(text)
            
            # If successful, return the result immediately
            return result

        except Exception as e:
            print(f"[AI Log] Model {model_name} failed: {e}")
            # Continue to the next model in the list
            continue

    print("[AI Error] All models failed. Returning None (fallback to math logic).")
    return None

def list_plants(username: str) -> list[dict]:
    """
    List all plants for the given user.

    Returns:
        List of dicts: [{plant_id, name, species, image_url, image_path, created_at}, ...]
    """
    username = _clean(username)
    if not username:
        return []

    db = get_db()
    ref = db.collection("users").document(username).collection("plants")

    try:
        snap = ref.order_by("created_at").stream()
    except Exception:
        snap = ref.stream()

    return [d.to_dict() for d in snap]


def delete_plant(username: str, plant_id: str) -> tuple[bool, str]:
    """
    Delete a plant document for the user.

    Args:
        username: Owner username
        plant_id: Plant id

    Returns:
        (ok, message)
    """
    username = _clean(username)
    plant_id = _clean(plant_id)

    if not username:
        return False, "Missing username (please login)."
    if not plant_id:
        return False, "Missing plant_id."

    db = get_db()
    try:
        db.collection("users").document(username).collection("plants").document(plant_id).delete()
        return True, "Deleted."
    except Exception as e:
        return False, f"Failed to delete plant: {e}"


def count_plants(username: str) -> int:
    """
    Count how many plants a user has.
    Uses aggregation count() if available, otherwise streams and counts.
    """
    username = _clean(username)
    if not username:
        return 0

    db = get_db()
    ref = db.collection("users").document(username).collection("plants")

    try:
        agg = ref.count().get()
        return int(agg[0].value)
    except Exception:
        return sum(1 for _ in ref.stream())


import io
from firebase_admin import storage

def add_plant_with_image(
    username: str,
    name: str,
    species: str = "",
    pil_image=None,
) -> tuple[bool, str]:
    """
    Create a new plant AND upload the image to Firebase Storage.
    No local files are saved.
    
    Path: user_uploads/{username}/{timestamp}_{uuid}.png
    """
    username = _clean(username)
    name = _clean(name)
    species = _clean(species)

    if not username:
        return False, "Missing username (please login)."
    if not name:
        return False, "Plant name is required."
    if pil_image is None:
        return False, "Missing image."

    try:
        # 1. Convert PIL image to bytes
        img_byte_arr = io.BytesIO()
        pil_image.save(img_byte_arr, format='PNG')
        img_bytes = img_byte_arr.getvalue()
        
        # 2. Prepare Cloud Storage Path
        # Naming: user_uploads/alice/20231220-123045_a1b2c3d4.png
        ts_str = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%S")
        unique_id = uuid.uuid4().hex[:8]
        blob_path = f"user_uploads/{username}/{ts_str}_{unique_id}.png"
        
        # 3. Upload to Firebase Storage
        bucket = storage.bucket() # Uses the bucket configured in config.py
        blob = bucket.blob(blob_path)
        
        print(f"[Storage] Uploading to {blob_path}...")
        blob.upload_from_string(img_bytes, content_type="image/png")
        
        # 4. Make Public and Get URL
        blob.make_public()
        public_url = blob.public_url
        print(f"[Storage] Success! URL: {public_url}")

    except Exception as e:
        print(f"[Error] Storage upload failed: {e}")
        return False, f"Failed to upload image: {e}"

    # 5. Save metadata to Firestore (using existing add_plant)
    return add_plant(
        username=username,
        name=name,
        species=species,
        image_path="",      # No local path
        image_url=public_url,
    )


### Cell 8: data_manager.py

In [ ]:
%%writefile data_manager.py

from __future__ import annotations

import nltk
import time
import json
import requests
import os
import glob
import re
import concurrent.futures
from config import get_db as _get_central_db
import datetime as _dt
from typing import Optional, List, Dict, Any
from collections import defaultdict

import firebase_admin
from firebase_admin import firestore

from nltk.stem import PorterStemmer


try:
    from docx import Document
except ImportError:
    Document = None
    print("Warning: python-docx not installed. Run: pip install python-docx")

# ==========================================
# SETUP
# ==========================================

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Downloading NLTK stopwords...")
    nltk.download('stopwords')
    nltk.download('punkt')

SENSORS_COL = "sensors"
ARTICLES_COL = "articles"

def get_db():
    return _get_central_db()

def _server_ts():
    try:
        return firestore.SERVER_TIMESTAMP
    except Exception:
        return _dt.datetime.utcnow().isoformat()

def _doc_to_dict(doc):
    d = doc.to_dict() if hasattr(doc, "to_dict") else dict(doc)
    d["id"] = doc.id if hasattr(doc, "id") else d.get("id")
    for k in ("created_at", "updated_at", "timestamp"):
        v = d.get(k)
        if hasattr(v, "isoformat"):
            d[k] = v.isoformat()
    return d

# ==========================================
# SENSORS (IoT)
# ==========================================

def add_sensor_reading(plant_id, temp=None, humidity=None, soil=None, light=None, extra=None):
    db = get_db()
    payload = {
        "plant_id": plant_id,
        "timestamp": _server_ts(),
        "created_at": _server_ts(),
    }
    if temp is not None:     payload["temp"] = float(temp)
    if humidity is not None: payload["humidity"] = float(humidity)
    if soil is not None:     payload["soil"] = float(soil)
    if light is not None:    payload["light"] = float(light)
    if extra:                payload.update(extra)

    ref = db.collection(SENSORS_COL).add(payload)[1]
    return ref.id

def get_sensor_history(plant_id: str, limit: Optional[int] = None) -> List[Dict[str, Any]]:
    db = get_db()
    q = (db.collection(SENSORS_COL)
           .where("plant_id", "==", plant_id)
           .order_by("timestamp", direction=firestore.Query.DESCENDING))
    if limit:
        q = q.limit(int(limit))
    return [_doc_to_dict(doc) for doc in q.stream()]

def get_latest_reading(plant_id: str) -> Optional[Dict[str, Any]]:
    rows = get_sensor_history(plant_id, limit=1)
    return rows[0] if rows else None

def get_all_readings(limit: Optional[int] = None) -> List[Dict[str, Any]]:
    db = get_db()
    q = db.collection(SENSORS_COL).order_by("timestamp", direction=firestore.Query.DESCENDING)
    if limit:
        q = q.limit(int(limit))
    return [_doc_to_dict(doc) for doc in q.stream()]

# ==========================================
# EXTERNAL IOT SERVER INTEGRATION
# ==========================================

IOT_SERVER_URL = "https://server-cloud-v645.onrender.com/history"

def sync_iot_data(plant_id: str) -> bool:
    print("--- Connecting to IoT Server... (Parallel Fetch) ---")

    feeds = ["temperature", "humidity", "soil"]
    sensor_data: Dict[str, float] = {}

    def _fetch(feed):
        try:
            # Short timeout to ensure speed
            resp = requests.get(IOT_SERVER_URL, params={"feed": feed, "limit": 1}, timeout=5)
            if resp.status_code == 200:
                d = resp.json()
                if "data" in d and len(d["data"]) > 0:
                    val = float(d["data"][0]["value"])
                    print(f"[IOT] Fetched {feed}: {val}")
                    return feed, val
        except Exception as e:
            print(f"[IOT] Err {feed}: {e}")
        return feed, None

    try:
        with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
            # Start all requests
            futures = [executor.submit(_fetch, f) for f in feeds]
            
            # Collect results
            for future in concurrent.futures.as_completed(futures):
                f, val = future.result()
                if val is not None:
                    sensor_data[f] = val

        if sensor_data:
            final_temp = sensor_data.get("temperature")
            final_hum = sensor_data.get("humidity")
            final_soil = sensor_data.get("soil")

            new_id = add_sensor_reading(
                plant_id,
                temp=final_temp,
                humidity=final_hum,
                soil=final_soil
            )
            print(f"[IOT] Data synced to Firestore successfully. ID: {new_id}")
            return True

        return False

    except Exception as e:
        print(f"[IOT] Connection failed: {e}")
        return False

# ==========================================
# ARTICLES (TXT/DOCX) + CRUD
# ==========================================


def extract_article_metadata(title: str, content: str, url: str | None = None) -> dict:
    """
    Best-effort metadata extraction (simple, lecturer-friendly).
    Returns keys: authors, journal, year, doi
    """
    text = (content or "")
    t = (title or "")

    # DOI (common pattern)
    doi_match = re.search(r"\b10\.\d{4,9}/[-._;()/:A-Za-z0-9]+\b", text)
    doi = doi_match.group(0) if doi_match else None

    # Year (pick first reasonable year)
    year_match = re.search(r"\b(19\d{2}|20\d{2})\b", text)
    year = year_match.group(0) if year_match else None

    # Journal (very heuristic)
    journal = None
    j_match = re.search(r"(?:Journal|Proceedings|Conference)\s*[:\-]\s*([^\n\r]{3,120})", text, re.IGNORECASE)
    if j_match:
        journal = j_match.group(1).strip()

    # Authors (heuristic)
    authors = None
    a_match = re.search(r"(?:Authors?)\s*[:\-]\s*([^\n\r]{3,180})", text, re.IGNORECASE)
    if a_match:
        authors = a_match.group(1).strip()

    # Fallback: if title looks like "X et al 2022" etc.
    if not authors:
        etal = re.search(r"([A-Z][A-Za-z\-]+)\s+et\s+al\.?", t)
        if etal:
            authors = f"{etal.group(1)} et al."

    meta = {
        "authors": authors,
        "journal": journal,
        "year": year,
        "doi": doi,
    }

    # optional: store url too (not required)
    if url:
        meta["url"] = url

    # clean Nones
    return {k: v for k, v in meta.items() if v}



def read_text_from_file(file_path):
    """Reads text from a file (.docx or .txt)."""
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".docx":
        try:
            doc = Document(file_path)
            full_text = []
            for para in doc.paragraphs:
                full_text.append(para.text)
            return "\n".join(full_text)
        except Exception as e:
            print(f"Error reading DOCX {file_path}: {e}")
            return ""

    else:
        try:
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                return f.read()
        except Exception as e:
            print(f"Error reading TXT {file_path}: {e}")
            return ""

def add_article(title: str, content: str, url: Optional[str] = None, metadata: Optional[dict] = None):
    db = get_db()

    existing = db.collection(ARTICLES_COL).where("title", "==", title).limit(1).stream()
    if list(existing):
        print(f"Skipping duplicate: {title}")
        return None

    
    auto_meta = extract_article_metadata(title, content, url)
    final_meta = {**auto_meta, **(metadata or {})}

    doc = {
        "title": title,
        "content": content,
        "url": url,
        "metadata": final_meta,
        "created_at": _server_ts(),
        "updated_at": _server_ts(),
    }

    ref = db.collection(ARTICLES_COL).add(doc)[1]
    return ref.id


def get_all_articles(limit: Optional[int] = None):
    db = get_db()
    q = db.collection(ARTICLES_COL).order_by("created_at", direction=firestore.Query.DESCENDING)
    if limit is not None:
        q = q.limit(int(limit))
    return [_doc_to_dict(doc) for doc in q.stream()]

def get_article_by_id(article_id: str):
    db = get_db()
    doc = db.collection(ARTICLES_COL).document(article_id).get()
    return _doc_to_dict(doc) if doc.exists else None

def add_article_from_txt(file_path: str, title: str, url: Optional[str] = None, metadata: Optional[dict] = None):
    text = read_text_from_file(file_path)
    if not text.strip():
        print(f"Skipped empty/unreadable file: {file_path}")
        return None
    return add_article(title=title, content=text, url=url, metadata=metadata)




#=============================================
#INDEX (term + DocIDs)
#=============================================




stemmer = PorterStemmer()

INDEX_COL = "index"

STOPWORDS = {
    "a","an","the","and","or","in","on","at","of","for","to",
    "is","are","as","by","with","from","this","that","it","be","was","were",
    "which","how","what","where","when","who","can","will","not","but",
    "has","have","had","do","does","did"
}

def _tokenize(text: str):
    if not text:
        return []
    return re.findall(r"\w+", text.lower())

def _normalize(tokens, use_stem: bool = True):
    out = []
    for t in tokens:
        if len(t) < 3:
            continue
        if t in STOPWORDS:
            continue
        if t.isdigit():
            continue
        if use_stem:
            try:
                t = stemmer.stem(t)
            except Exception:
                pass
        out.append(t)
    return out

def build_index(max_docs: int = 5, use_stem: bool = True, include_title: bool = True):
    db = get_db()
    
    # [OPTIMIZATION] Skip if index already exists
    try:
        # Check if we have at least one document in the 'index' collection
        if next(db.collection(INDEX_COL).limit(1).stream(), None):
            print(f"[OPTIMIZATION] Index '{INDEX_COL}' already exists. Skipping rebuild.")
            return {}
    except Exception:
        pass # If check fails, safe to proceed with build

    articles = get_all_articles(limit=max_docs)
    if not articles:
        print("No articles found. Seed articles first.")
        return {}

    docnum_to_articleid: Dict[str, str] = {}
    numbered = []
    for i, a in enumerate(articles, start=1):
        doc_num = f"doc_{i}"
        docnum_to_articleid[doc_num] = a["id"]
        numbered.append((doc_num, a))

    term_to_docids = defaultdict(set)

    for doc_num, a in numbered:
        title = a.get("title", "") if include_title else ""
        content = a.get("content", "")
        text = (title + " " + content).strip()

        tokens = _tokenize(text)
        terms = _normalize(tokens, use_stem=use_stem)

        for term in set(terms):
            term_to_docids[term].add(doc_num)

    col = db.collection(INDEX_COL)

    # clear previous index
    batch = db.batch()
    ops = 0
    for d in col.stream():
        batch.delete(d.reference)
        ops += 1
        if ops >= 400:
            batch.commit()
            batch = db.batch()
            ops = 0
    if ops:
        batch.commit()

    # write required schema
    batch = db.batch()
    ops = 0
    for term, docids_set in term_to_docids.items():
        ref = col.document(term)
        batch.set(ref, {
            "term": term,
            "DocIDs": sorted(docids_set),
        })
        ops += 1
        if ops >= 400:
            batch.commit()
            batch = db.batch()
            ops = 0
    if ops:
        batch.commit()

    print(f"Built index in '{INDEX_COL}' with {len(term_to_docids)} terms.")
    print("DocIDs mapping:")
    for k, v in docnum_to_articleid.items():
        print(f"  {k} -> article_id={v}")

    return docnum_to_articleid





#============================
# RAG
#============================

import numpy as np

# Dependency checks (like lecturer)
try:
    import chromadb
    CHROMADB_AVAILABLE = True
except ImportError:
    CHROMADB_AVAILABLE = False

try:
    from sentence_transformers import SentenceTransformer
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    TRANSFORMERS_AVAILABLE = False

try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False

try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False
    print("Warning: google-generativeai not installed. Run: pip install google-generativeai")


class SimpleVectorStore:
    """Fallback vector store when ChromaDB is not available."""
    def __init__(self):
        self.documents = []
        self.embeddings = []
        self.metadatas = []
        self.ids = []

    def add(self, embeddings, documents, metadatas, ids):
        self.embeddings.extend(list(embeddings))
        self.documents.extend(list(documents))
        self.metadatas.extend(list(metadatas))
        self.ids.extend(list(ids))

    def query(self, query_embeddings, n_results=5):
        if not self.embeddings:
            return {'ids':[[]], 'documents':[[]], 'metadatas':[[]], 'distances':[[]]}

        X = np.array(self.embeddings, dtype=np.float32)
        q = np.array(query_embeddings, dtype=np.float32)

        # cosine similarity
        q_norm = np.linalg.norm(q, axis=1, keepdims=True) + 1e-12
        X_norm = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
        qn = q / q_norm
        Xn = X / X_norm
        sims = (Xn @ qn.T).reshape(-1)

        top = np.argsort(sims)[::-1][:n_results]
        distances = [float(1 - sims[i]) for i in top]  # 1 - similarity

        return {
            "ids": [[self.ids[i] for i in top]],
            "documents": [[self.documents[i] for i in top]],
            "metadatas": [[self.metadatas[i] for i in top]],
            "distances": [distances],
        }

    def count(self):
        return len(self.documents)


class PlantRAG:
    """
    Lecturer-style RAG:
    - Embeddings: SentenceTransformer OR TF-IDF fallback
    - Vector store: ChromaDB OR SimpleVectorStore fallback
    - Generation: OpenAI OR Template fallback
    """

    def __init__(self, google_api_key: str | None = None):
        # Embeddings setup
        self.use_transformers = False
        self.use_tfidf = False
        self.fitted = False

        if TRANSFORMERS_AVAILABLE:
            try:
                self.embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
                self.use_transformers = True
            except Exception:
                self.use_transformers = False

        if (not self.use_transformers):
            if not SKLEARN_AVAILABLE:
                raise RuntimeError("No SentenceTransformers and no sklearn TF-IDF available. Install sentence-transformers or scikit-learn.")
            self.tfidf = TfidfVectorizer(max_features=2000, stop_words="english")
            self.use_tfidf = True

        # Vector store setup
        self.use_chromadb = False
        if CHROMADB_AVAILABLE:
            try:
                client = chromadb.Client()
                try:
                    self.collection = client.get_collection("plant_articles")
                except Exception:
                    self.collection = client.create_collection("plant_articles")
                self.use_chromadb = True
            except Exception:
                self.collection = SimpleVectorStore()
                self.use_chromadb = False
        else:
            self.collection = SimpleVectorStore()
            self.use_chromadb = False

        # --- GEMINI SETUP ---
        self.use_gemini = False
        
        # Priority list for models
        self.models_to_try = [
            'gemini-2.0-flash',       # First priority
            'gemini-1.5-flash',       # Stable/Fast
            'gemini-pro',             # Classic
            'gemini-flash-latest'     # Fallback generic
        ]

        # Try to get key from args OR environment variable
        api_key = google_api_key or os.environ.get("GOOGLE_API_KEY")

        if api_key and GEMINI_AVAILABLE:
            try:
                genai.configure(api_key=api_key)
                self.use_gemini = True
            except Exception as e:
                print(f"Gemini config error: {e}")
                self.use_gemini = False
        
        self.loaded = False


    def preprocess_text(self, text: str) -> str:
        if not text:
            return ""
        text = re.sub(r"\s+", " ", text)
        return text.strip()


    def generate_embeddings(self, texts: list[str]):
        if self.use_transformers:
            return self.embedding_model.encode(texts, show_progress_bar=False)

        # TF-IDF fallback
        if not self.fitted:
            self.tfidf.fit(texts)
            self.fitted = True
        return self.tfidf.transform(texts).toarray()


    def load_from_firestore(self, limit: int | None = None):
        if self.loaded:
            return 0

        papers = get_all_articles(limit=limit)
        documents, metadatas, ids = [], [], []

        for i, a in enumerate(papers):
            title = a.get("title", "Unknown")
            content = a.get("content", "")
            text = self.preprocess_text(f"{title}\n{content}")
            if len(text) < 30:
                continue

            documents.append(text)
            metadatas.append({
            "title": title,
            "article_id": a.get("id"),
            "url": a.get("url"),
            "metadata": a.get("metadata") or {},
            })

            ids.append(f"article_{i}")

        if not documents:
            raise RuntimeError("No articles with content found in Firestore.")

        emb = self.generate_embeddings(documents)


        if self.use_chromadb:
            try:
                existing = self.collection.get()
                if existing and existing.get("ids"):
                    self.collection.delete(ids=existing["ids"])
            except Exception:
                pass


        if self.use_chromadb:
            self.collection.add(
                embeddings=[e.tolist() for e in emb],
                documents=documents,
                metadatas=metadatas,
                ids=ids
            )
        else:
            self.collection.add(
                embeddings=emb,
                documents=documents,
                metadatas=metadatas,
                ids=ids
            )

        self.loaded = True
        return len(documents)


    def search(self, query: str, n_results: int = 5):
        q = self.preprocess_text(query)
        q_emb = self.generate_embeddings([q])

        if self.use_chromadb:
            return self.collection.query(query_embeddings=q_emb.tolist(), n_results=n_results)
        else:
            return self.collection.query(query_embeddings=q_emb, n_results=n_results)


    def _template_response_smart(self, question: str, docs: list[str]) -> str:
        """
        Technical Fallback: Extracts a relevant text window around the search term.
        This runs when all AI models fail to generate a response.
        """
        if not docs:
            return "No relevant documents found."
            
        # Identify the first keyword (simple heuristic for text search)
        key_term = question.split()[0].lower() if question else ""
        
        # Default snippet: the beginning of the first document
        best_snippet = docs[0][:400] + "..." 
        
        found_better = False
        if key_term and len(key_term) > 2:
            for doc in docs:
                # Search for the keyword within the article content
                idx = doc.lower().find(key_term)
                if idx != -1:
                    # Found the term! Slice a context window around it (approx. 300 chars)
                    start = max(0, idx - 50)
                    end = min(len(doc), idx + 350)
                    best_snippet = "..." + doc[start:end] + "..."
                    found_better = True
                    break
        
        prefix = "🤖 **AI Service Unavailable.**\nHere is a relevant excerpt from your library:\n\n"
        return prefix + best_snippet

    def generate_response(self, question: str, docs: list[str], metas: list[dict], sims: list[float]) -> str:
        """
        Generates an answer using Gemini with a Model Cascade strategy.
        If all Gemini models fail, falls back to smart snippet extraction.
        """
        
        if not self.use_gemini:
            return self._template_response_smart(question, docs)

    
        context_text = ""
        for m, d in zip(metas, docs):
            title = m.get('title', 'Unknown Source')
            context_text += f"\n---\nTitle: {title}\nExcerpt: {d[:1000]}...\n"

        prompt = f"""
        You are an expert botanist assistant for a smart garden system.
        
        Instructions:
        1. Answer the user's question based PRIMARILY on the provided Context Articles below.
        2. If the answer is found in the context, cite the title.
        3. If the answer is NOT in the context, you MUST use your general knowledge to answer, but start your sentence with: "Based on general agricultural knowledge (not from your library)..."
        4. Keep the answer concise (max 3-4 sentences) and helpful.

        User Question: {question}

        Context Articles:
        {context_text}
        """

        for model_name in self.models_to_try:
            try:
                # print(f"Trying model: {model_name}...") # Debug line
                model = genai.GenerativeModel(model_name)
                response = model.generate_content(prompt)
                
                if response and response.text:
                    return response.text 
                    
            except Exception as e:
                print(f"Model {model_name} failed: {e}")
                continue

        print("All Gemini models failed. Switching to Technical Fallback.")
        return self._template_response_smart(question, docs)


    def query(self, question: str, top_k: int = 5, fallback_threshold: float = 0.20) -> dict:
        if not self.loaded:
            self.load_from_firestore()

        res = self.search(question, n_results=top_k)

        docs = res["documents"][0]
        metas = res["metadatas"][0]
        dists = res["distances"][0]

        if not docs:
            return {
                "response": "No results found. Try different keywords.",
                "papers_found": 0,
                "used_fallback": True,
                "best_sim": 0.0,
                "sources": [],
                "chunks": []
            }

        # Convert distances to similarities (1 - distance)
        sims = []
        for d in dists:
            try:
                d = float(d)
            except Exception:
                d = 999.0
            sim = 1.0 - d
            sims.append(sim)

        best_sim = max(sims) if sims else 0.0
        used_fallback = best_sim < fallback_threshold

        # Build chunks for UI
        chunks = []
        for m, doc, sim in zip(metas, docs, sims):
            chunks.append({
                "title": m.get("title") or "Untitled",
                "url": m.get("url"),
                "snippet": (doc[:320].replace("\n", " ") + "...") if doc else "",
                "article_id": m.get("article_id"),
                "metadata": (m.get("metadata") or {}),
            })

        response_text = self.generate_response(question, docs, metas, sims)

        return {
            "response": response_text,
            "papers_found": len(docs),
            "used_fallback": used_fallback,
            "best_sim": float(best_sim),
            "sources": metas,
            "chunks": chunks
        }



# SEED ARTICLES FROM LOCAL FOLDER + BUILD INDEX


def seed_database_with_articles(folder_path: str = "articles_data", do_build_index: bool = True):

    print("--- Seeding Database with REAL Data ---")

    if not os.path.exists(folder_path):
        print(f"Warning: Folder '{folder_path}' not found. Creating it now...")
        os.makedirs(folder_path)
        print(f"Please put your .docx/.txt files in '{folder_path}' and restart.")
        return

    files = glob.glob(os.path.join(folder_path, "*.docx")) + glob.glob(os.path.join(folder_path, "*.txt"))
    if not files:
        print(f"No files found in {folder_path}.")
        return

    print(f"Found {len(files)} files. Processing...")
    for file_path in files:
        filename = os.path.basename(file_path)
        title = os.path.splitext(filename)[0].replace("_", " ").title()

        print(f"Reading: {filename}...")
        content = read_text_from_file(file_path)
        if content:
            add_article(title=title, content=content)
        else:
            print(f"Skipped empty or unreadable file: {filename}")

    if do_build_index:
        print("Building Index")
        build_index(max_docs=5, use_stem=True)


def generate_vacation_report(username, days_away):
    """
    Generates a survival report utilizing AI for context-aware predictions 
    (Temperature/Plant Type). Falls back to mathematical logic if AI fails.
    
    Args:
        username (str): The user requesting the report.
        days_away (int): Number of days the user will be away.
        
    Returns:
        list: A list of report rows [Plant Name, Current Soil, Status, Message].
    """ 
    import plants_manager
    from plants_manager import list_plants, get_vacation_advice_ai

    user_plants = plants_manager.list_plants(username)
    report = []
    
    # Global safety limit for vacations (in days)
    GLOBAL_MAX_DAYS = 21 

    for plant in user_plants:
        plant_id = plant.get("plant_id")
        plant_name = plant.get("name", "Unknown Plant")
        plant_threshold = plant.get('min_soil', 30)

        # 1. Fetch real-time sensor data
        latest_data = get_latest_reading(plant_id)
        
        # Default values if no sensor data is found
        current_soil = float(latest_data.get("soil", 0)) if latest_data else 0
        # Default to 25°C if no temp sensor available
        current_temp = float(latest_data.get("temp", 25)) if latest_data else 25 

        try:
            days = int(days_away)
        except (ValueError, TypeError):
            days = 0

        status = ""
        msg = ""
        
        # === AI ENHANCEMENT START ===
        # Attempt to get smart advice based on temperature and plant species
        ai_result = get_vacation_advice_ai(
            plant_name=plant_name,
            current_soil=current_soil,
            min_threshold=plant_threshold,
            current_temp=current_temp,
            days_away=days
        )
        
        if ai_result:
            # If AI analysis is successful, use its recommendation
            status = ai_result.get("status", "UNKNOWN")
            
            # Combine the explanation and the actionable recommendation
            msg = f"{ai_result.get('message')} -> {ai_result.get('recommendation')}"
            
            # Add visual indicators based on status
            if status == "CRITICAL": status += " 💀"
            elif status == "NEEDS WATER": status += " 💧"
            elif status == "SAFE": status += " ✅"

        else:
            # === FALLBACK LOGIC ===
            # Used if AI fails or API key is missing. 
            # Basic mathematical estimation based on plant thirst level.
            print(f"[Fallback] Using math logic for {plant_name}")
            
            # Estimate drying rate based on threshold (thirstier plants dry faster)
            if plant_threshold > 20:
                drying_rate = 10.0  # Fast drying (approx. 10% per day)
            else:
                drying_rate = 2.0   # Slow drying (approx. 2% per day)

            predicted_soil = current_soil - (days * drying_rate)
            
            if days > GLOBAL_MAX_DAYS:
                status = "CRITICAL 💀"
                msg = "Vacation too long. System limit exceeded."
            elif predicted_soil < plant_threshold:
                status = "NEEDS WATER 💧"
                # Calculate how many days until critical level
                days_left = max(0, int((current_soil - plant_threshold) / drying_rate))
                msg = f"Will dry in {days_left} days. Water or add irrigation."
            else:
                status = "SAFE ✅"
                msg = f"Predicted soil: {int(predicted_soil)}%. Have fun!"
        
        # === END PROCESS ===

        report.append([plant_name, f"{current_soil}%", status, msg])

    return report

### Cell 9: logic_handler.py

In [ ]:
%%writefile logic_handler.py

import config
import auth_service
import gamification_rules
import data_manager

def handle_gamified_action(username, action_key, plant_id=None):
    """
    Executes a gamified action, updates scores, triggers IoT/sync if needed,
    and returns a summary string of what happened.
    """
    messages = []
    
    # 1. Score Update
    points = gamification_rules.get_points_for_action(action_key)
    if points > 0:
        new_score = auth_service.update_user_score(username, points)
        messages.append(f"Action recorded! You earned {points} points. New Score: {new_score}")
    
    # 2. IoT Sync for physical actions
    physical_actions = ["WATER_PLANT", "FERTILIZE_PLANT"]
    if action_key in physical_actions and plant_id:
        # Trigger IoT sync
        success = data_manager.sync_iot_data(plant_id)
        if success:
            messages.append("Plant environment updated with REAL data (IoT Synced).")
        else:
            messages.append("Failed to fetch real data (IoT Sync failed).")
            
    # 3. Weekly Challenge Update
    status = auth_service.update_weekly_challenge_progress(username, action_key)
    if status and status.get('relevant'):
        progress_str = f"Challenge Progress: {status['progress']}/{status['target']}"
        messages.append(progress_str)
        
        if status.get('completed'):
            bonus = status['bonus_awarded']
            if bonus > 0:
                messages.append(f"CHALLENGE COMPLETED! Bonus {bonus} points awarded!")
            else:
                messages.append("Challenge already completed previously.")
                
    return "\n".join(messages)


### Cell 10: main.py

In [ ]:
%%writefile main.py

import config
import data_manager
from ui.home_ui import home_screen
import threading
import time

# ==========================================
# BACKGROUND AUTO-FETCHER
# ==========================================

AUTO_FETCH_INTERVAL_SECONDS = 600  # 10 minutes

def _get_all_plant_ids():
    """Returns all plant_id strings from all users."""
    db = config.get_db()
    plant_ids = []
    
    try:
        # Get all users
        users = db.collection("users").stream()
        for user_doc in users:
            username = user_doc.id
            # Get all plants for this user
            plants = db.collection("users").document(username).collection("plants").stream()
            for plant_doc in plants:
                plant_data = plant_doc.to_dict()
                pid = plant_data.get("plant_id")
                if pid:
                    plant_ids.append(pid)
    except Exception as e:
        print(f"[AutoFetch] Error fetching plant IDs: {e}")
    
    return plant_ids


def _auto_fetch_loop():
    """Background loop - syncs IoT data every 10 minutes."""
    print("[AutoFetch] Background scheduler started (10-minute interval)")
    
    while True:
        # Sleep first to allow app to fully initialize
        time.sleep(AUTO_FETCH_INTERVAL_SECONDS)
        
        print("[AutoFetch] Waking up... syncing IoT data for all plants")
        
        try:
            plant_ids = _get_all_plant_ids()
            
            if not plant_ids:
                print("[AutoFetch] No plants found in database")
                continue
            
            print(f"[AutoFetch] Found {len(plant_ids)} plants. Syncing...")
            
            for plant_id in plant_ids:
                try:
                    data_manager.sync_iot_data(plant_id)
                except Exception as e:
                    print(f"[AutoFetch] Error syncing plant {plant_id}: {e}")
            
            print(f"[AutoFetch] Sync complete. Next sync in {AUTO_FETCH_INTERVAL_SECONDS // 60} minutes.")
            
        except Exception as e:
            print(f"[AutoFetch] Error in sync cycle: {e}")


def start_background_scheduler():
    """Starts daemon thread for auto-fetching."""
    thread = threading.Thread(target=_auto_fetch_loop, daemon=True)
    thread.start()
    print("[AutoFetch] Background thread initialized")
    return thread


# ==========================================
# MAIN APPLICATION
# ==========================================

def main():
    print("--- SHARK TEAM CLOUD SYSTEM - GUI MODE ---")
    
    # 1. Initialize Infrastructure
    try:
        db = config.get_db()
        print("[OK] Database Connected")
        
        # 2. Setup/Seed Data (if needed)
        try:
            data_manager.seed_database_with_articles()
            print("[OK] Knowledge Base Ready")
        except Exception as seed_err:
            print(f"[WARNING] Knowledge Base seeding failed (likely quota/rate limit): {seed_err}")
            print("[SYSTEM] Continuing launch... RAG search might use existing index.")


    except Exception as e:
        print(f"[ERROR] Initialization failed: {e}")
        return

    # 3. Start Background Auto-Fetcher
    start_background_scheduler()

    # 4. Launch the Graphical Interface
    print("[SYSTEM] Launching User Interface...")
    app = home_screen()
    app.launch()

if __name__ == "__main__":
    main()

### Cell 11: app.py

In [ ]:
%%writefile app.py
from ui.home_ui import home_screen

app = home_screen()

if __name__ == "__main__":
    app.launch(share=True)


### Cell 12: ui/__init__.py

In [ ]:
%%writefile ui/__init__.py
# ui package


### Cell 13: ui/auth_ui.py

In [ ]:
%%writefile ui/auth_ui.py


import gradio as gr
from auth_service import register_user, login_user

def auth_screen(user_state: gr.State):
 
    gr.Markdown("##  Login / Register")

    mode = gr.Radio(
        choices=["Login", "Register"],
        value="Login",
        label="",
        interactive=True,
    )

    # -------- Login UI --------
    with gr.Column(visible=True) as login_col:
        login_username = gr.Textbox(label="Username")
        login_password = gr.Textbox(label="Password", type="password")
        login_btn = gr.Button("Login", variant="primary")
        login_msg = gr.Markdown()

    # -------- Register UI --------
    with gr.Column(visible=False) as reg_col:
        reg_username = gr.Textbox(label="Username (no spaces)")
        reg_display = gr.Textbox(label="Display name")
        reg_email = gr.Textbox(label="Email")
        reg_password = gr.Textbox(label="Password (min 6)", type="password")
        reg_btn = gr.Button("Create account")
        reg_msg = gr.Markdown()

    gr.Markdown("---")
    current_user = gr.Markdown("Not logged in.")

    # -------- Switching handler --------
    def switch(m):
        return gr.update(visible=(m == "Login")), gr.update(visible=(m == "Register"))
    
    mode.change(fn=switch, inputs=[mode], outputs=[login_col, reg_col])

    # ---------- handlers ----------
    def do_login(u, p):
    
        ok, res = login_user(u, p)
        if ok:
            # res is user_data dict (from Firestore)
            username = res.get("username", u)
            # Success: clear login fields
            return username, f"✅ Logged in as (`{username}`)", f"✅ Welcome back!", "", ""
        # Error: keep fields for retry
        return None, "Not logged in.", f"❌ {res}", gr.update(), gr.update()

    def do_register(u, d, pw, em):
        ok, msg = register_user(u, d, pw, em)
        if ok:
            # Success: clear all registration fields
            
            return (
                f"✅ {msg}<br>Now please login.",
                "", "", "", "",                 # clear register fields
                gr.update(value="Login"),       # switch mode to Login
                gr.update(visible=True),        # show login col
                gr.update(visible=False),       # hide register col
                (u or ""),                      # prefill login username
                ""                              # clear login password
            )
        return (
            f"❌ {msg}",
            gr.update(), gr.update(), gr.update(), gr.update(),
            gr.update(), gr.update(), gr.update(), gr.update(), gr.update()
        )

    # Login event is returned to home_ui.py
    login_event = login_btn.click(
        fn=do_login,
        inputs=[login_username, login_password],
        outputs=[user_state, current_user, login_msg, login_username, login_password],
    )
    # Register event updates tab selection + prefill login username
    reg_btn.click(
        fn=do_register,
        inputs=[reg_username, reg_display, reg_password, reg_email],
        outputs=[
            reg_msg,
            reg_username, reg_display, reg_password, reg_email,
            mode,
            login_col, reg_col,
            login_username, login_password
        ],
    )

    return login_event



### Cell 14: ui/home_ui.py

In [ ]:
%%writefile ui/home_ui.py

import gradio as gr
from datetime import datetime, timezone

from plants_manager import count_plants
from auth_service import logout_user
from data_manager import get_all_readings

from ui.plants_ui import plants_screen
from ui.sensors_ui import sensors_screen
from ui.search_ui import search_screen
from ui.upload_ui import upload_screen
from ui.dashboard_ui import dashboard_screen
from ui.auth_ui import auth_screen


# =========================
# Vacation mode bridge
# =========================
def run_vacation_check(days, current_username):
    if days is None:
        return []

    if not current_username:
        return [["Error", "-", "❌", "No user logged in"]]

    from data_manager import generate_vacation_report
    return generate_vacation_report(current_username, days)


# =========================
# Helpers for metrics
# =========================
def _parse_iso(ts):
    if not ts:
        return None
    try:
        return datetime.fromisoformat(ts)
    except Exception:
        try:
            return datetime.fromisoformat(ts.replace("Z", "+00:00"))
        except Exception:
            return None


def _time_ago(dt: datetime | None) -> str:
    if not dt:
        return "n/a"

    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)

    now = datetime.now(timezone.utc)
    diff = now - dt
    mins = int(diff.total_seconds() // 60)

    if mins < 1:
        return "just now"
    if mins < 60:
        return f"{mins}m ago"

    hrs = mins // 60
    if hrs < 24:
        return f"{hrs}h ago"

    return f"{hrs // 24}d ago"


def _compute_overview_metrics(username=None):
    """
    Computes Home overview metrics:
    - number of plants
    - last sensor reading time
    - average soil moisture (last 50 readings)
    """
    try:
        plants_n = int(count_plants(username) if username else 0)
    except Exception:
        plants_n = 0

    try:
        latest = get_all_readings(limit=1) or []
        latest_ts = _parse_iso(latest[0].get("timestamp")) if latest else None
        last_reading = _time_ago(latest_ts)

        recent = get_all_readings(limit=50) or []
        soils = [r.get("soil") for r in recent if r.get("soil") is not None]
        avg_soil = round(sum(soils) / len(soils), 1) if soils else 0.0

    except Exception:
        last_reading, avg_soil = "n/a", 0.0

    return plants_n, last_reading, avg_soil

# =========================
# HOME SCREEN
# =========================
def home_screen():
    """
    Builds the main dashboard UI.
    Includes custom CSS to fix table formatting for the Vacation Mode report.
    """
    
    # Custom CSS to control the table layout:
    # 1. Enforces 'nowrap' on the 3rd column (Status) to keep the icon and text on one line.
    # 2. Sets a minimum width for readability.
    custom_css = """
    .vacation-table td:nth-child(3) { 
        white-space: nowrap !important; 
        min-width: 140px;
    }
    """

    with gr.Blocks(title="My Garden Care", theme=gr.themes.Glass(), css=custom_css) as app:
        user_state = gr.State(value=None)

        # ---------- TOP BAR ----------
        with gr.Row():
            gr.Markdown("## 🌿 My Garden Care")
            user_status_label = gr.Markdown("")
  
        # NAV BAR (hidden until login)
        with gr.Row(equal_height=True, visible=False)  as nav_row:
            btn_home = gr.Button("Home", variant="secondary", scale=1, min_width=140)
            btn_sensors = gr.Button("Sensors", variant="secondary", scale=1, min_width=140)
            btn_search = gr.Button("Search", variant="secondary", scale=1, min_width=140)
            btn_dashboard = gr.Button("Plant Dashboard", variant="secondary", scale=1, min_width=140)
            btn_upload = gr.Button("Upload a Photo", variant="secondary", scale=1, min_width=140)
            logout_btn = gr.Button("Logout", visible=False, scale=1, min_width=140)

        gr.Markdown("---")

        # ---------- HOME ----------
        with gr.Column(visible=False) as home:
            gr.Markdown("## Welcome")

            with gr.Row():
                qa_plants = gr.Button("🌿 View my plants", variant="secondary")

            with gr.Row():
                m_plants = gr.Number(label="My plants", interactive=False)
                m_last = gr.Textbox(label="Last sensor reading", interactive=False)
                m_avg_soil = gr.Number(label="Avg soil (last 50)", interactive=False)

            btn_refresh = gr.Button("Refresh", variant="secondary")

            

            # ---------- VACATION MODE ----------
            with gr.Accordion("✈️ Planning a vacation? Check your plants", open=False):
                gr.Markdown(
                    "Estimate whether your plants will survive while you're away, "
                    "based on real soil moisture data."
                )

                with gr.Row():
                    days_input = gr.Number(
                        label="Days Away",
                        value=None,
                        precision=0,
                        placeholder="e.g. 5"
                    )
                    check_btn = gr.Button("Check", variant="primary")

                # Table configuration:
                # - 'column_widths': Allocates 50% width to the Message column to prevent cramping.
                # - 'elem_classes': Links this component to the 'vacation-table' CSS class defined above.
                vacation_table = gr.Dataframe(
                    headers=["Plant", "Current Soil", "Status", "Message"],
                    interactive=False,
                    wrap=True,
                    column_widths=["15%", "10%", "15%", "60%"], 
                    elem_classes="vacation-table"
                )

                check_btn.click(
                    fn=run_vacation_check,
                    inputs=[days_input, user_state],
                    outputs=[vacation_table]
                )

        # ---------- OTHER PAGES ----------

        with gr.Column(visible=False) as plants:
            plants_btn, plants_load, plants_inputs, plants_outputs = plants_screen(user_state)

        with gr.Column(visible=False) as sensors:
            sensors_btn, sensors_load, sensors_inputs, sensors_outputs = sensors_screen(user_state)

        with gr.Column(visible=False) as search:
            search_screen()

        with gr.Column(visible=False) as dashboard:
            dashboard_btn, dashboard_load, dashboard_inputs, dashboard_outputs = dashboard_screen(user_state)

        with gr.Column(visible=False) as upload:
            upload_screen(user_state)

        with gr.Column(visible=True) as auth:
            login_event = auth_screen(user_state)

        # ---------- NAV ----------
        def go(target):
            return [
                gr.update(visible=(target == "home")),
                gr.update(visible=(target == "plants")),
                gr.update(visible=(target == "sensors")),
                gr.update(visible=(target == "search")),
                gr.update(visible=(target == "dashboard")),
                gr.update(visible=(target == "upload")),
                gr.update(visible=(target == "auth")),
            ]

        pages = [home, plants, sensors, search, dashboard, upload, auth]
        
        # Always start on Auth page 
        app.load(lambda: go("auth"), outputs=pages)

        # ------------------------
        # Auto-load on navigation
        # ------------------------
        btn_home.click(lambda: go("home"), outputs=pages)
        btn_sensors.click(lambda: go("sensors"), outputs=pages).then(
            fn=sensors_load, inputs=sensors_inputs, outputs=sensors_outputs
        )
        btn_search.click(lambda: go("search"), outputs=pages)
        btn_dashboard.click(lambda: go("dashboard"), outputs=pages).then(
            fn=dashboard_load, inputs=dashboard_inputs, outputs=dashboard_outputs
        )
        btn_upload.click(lambda: go("upload"), outputs=pages)
        # btn_auth.click(lambda: go("auth"), outputs=pages)

        # btn_open_plants.click(lambda: go("plants"), outputs=pages)
        qa_plants.click(lambda: go("plants"), outputs=pages).then(
            fn=plants_load, inputs=plants_inputs, outputs=plants_outputs
        )

        # ---------- METRICS ----------
        def refresh_metrics(u):
            username = u.strip() if isinstance(u, str) else None
            return _compute_overview_metrics(username)

        btn_refresh.click(refresh_metrics, inputs=[user_state], outputs=[m_plants, m_last, m_avg_soil])
        app.load(refresh_metrics, inputs=[user_state], outputs=[m_plants, m_last, m_avg_soil])

        # ------------------------
        # Login -> show navbar + redirect to home
        # ------------------------
        def on_login_success(username):
            """
            After successful login:
            - show navbar
            - show logout button
            - set user status label
            - refresh home metrics
            """
            if username:
                plants_n, last_reading, avg_soil = _compute_overview_metrics(username)
                return (
                    gr.update(visible=True),               # nav_row
                    gr.update(visible=True),               # logout_btn
                    f"👤 Logged in as: **{username}**",     # user_status_label
                    plants_n, last_reading, avg_soil
                )

            return (
                gr.update(visible=False),
                gr.update(visible=False),
                "",
                0, "n/a", 0.0
            )

        login_event.then(
            fn=on_login_success,
            inputs=[user_state],
            outputs=[nav_row, logout_btn, user_status_label, m_plants, m_last, m_avg_soil]
        )

        # Redirect to Home after login
        login_event.then(lambda: go("home"), outputs=pages)

        # ------------------------
        # Logout -> hide navbar + go to auth
        # ------------------------
        def do_logout():
            logout_user()
            return (
                None,                        # user_state
                gr.update(visible=False),    # nav_row
                gr.update(visible=False),    # home
                gr.update(visible=False),    # plants
                gr.update(visible=False),    # sensors
                gr.update(visible=False),    # search
                gr.update(visible=False),    # dashboard
                gr.update(visible=False),    # upload
                gr.update(visible=True),     # auth
                gr.update(visible=True),     # logout_btn (inside hidden nav anyway)
                "",                          # user_status_label
            )

        logout_btn.click(
            fn=do_logout,
            outputs=[user_state, nav_row, home, plants, sensors, search, dashboard, upload, auth, logout_btn, user_status_label]
        )


    return app


### Cell 15: ui/plants_ui.py

In [ ]:
%%writefile ui/plants_ui.py

import gradio as gr
from plants_manager import list_plants, delete_plant


def _get_username(user_state):
    # Helper: Extract username from the shared user_state.
    # Args:
    #     user_state: expected to be a string username or None.
    # Returns:
    #     str: username if logged-in, else "".
    return user_state.strip() if isinstance(user_state, str) else ""


def plants_screen(user_state: gr.State):
  #   Plants gallery screen.
  #   The screen shows:
  #   - Gallery of plant images (previewable)
  #   - A delete dropdown + delete button (only visible when data exists)
  #   Args:
  #       user_state (gr.State): shared state holding username string or None.
  #   Returns:
  #       tuple:
  #           (refresh_btn, load_fn, inputs_list, outputs_list)

    gr.Markdown("## 🌿 My Plants")
    gr.Markdown("*Click on any plant image to view it in full size.*")
    info = gr.Markdown()
    # refresh_btn = gr.Button("Load Plants", variant="primary")
    # Manual fallback. We hide it visually by default (scale=0),
    # but keep it for resilience/debugging.
    refresh_btn = gr.Button("Load plants", variant="secondary", scale=0)

    empty_state = gr.HTML()

    # Elegant grid gallery with preview support
    gallery = gr.Gallery(
        label="",
        columns=3,
        rows=2,
        height=350,
        object_fit="scale-down",
        allow_preview=True,
        preview=True,
        show_label=False,
        visible=False,
    )

    with gr.Row(visible=False) as delete_row:
        plant_to_delete = gr.Dropdown(label="Delete plant (by name)", choices=[], value=None)
        del_btn = gr.Button("Delete", variant="stop")

    del_status = gr.Markdown()

    def load(u):
      #   Load plants for current user:
      #   - If not logged-in -> show 'login required'
      #   - If no plants -> show empty state
      #   - Else -> fill gallery + delete dropdown

        username = _get_username(u)

        # --- Not logged in ---
        if not username:
            return (
                "⚠️ Please login to see your plants.",
                '<div class="card"><h3>🔒 Login required</h3><p>Please login.</p></div>',
                gr.update(visible=False, value=[]),
                gr.update(visible=False),
                gr.update(choices=[], value=None),
                ""
            )

        plants = list_plants(username) or []

        # --- Logged in but no plants ---
        if not plants:
            return (
                f"Logged in as **{username}**",
                '<div class="card"><h3>🌱 No plants yet</h3><p>Go to <b>Upload</b> to add your first plant, then come back and press <b>Load Plants</b>.</p></div>',
                gr.update(visible=False, value=[]),
                gr.update(visible=False),
                gr.update(choices=[], value=None),
                ""
            )

        # --- Have plants ---
        items = []
        delete_choices = []

        for p in plants:
            pid = p.get("plant_id") or p.get("id") or ""
            name = (p.get("name") or p.get("species") or "").strip() or "Plant"
            img = p.get("image_url") or p.get("image_path")

            # Gallery can display local server paths OR real URLs
            if img:
                items.append((img, name))

            # Show NAME to user, but keep pid as value
            if pid:
                delete_choices.append((name, pid))

        if not items:
            return (
                f"Logged in as **{username}**",
                '<div class="card"><h3>🖼️ No images found</h3><p>Your plants exist, but they don’t have images/URLs yet.</p></div>',
                gr.update(visible=False, value=[]),
                gr.update(visible=True),
                gr.update(choices=delete_choices, value=None),
                ""
            )

        return (
            f"✅ Loaded **{len(items)}** plants.",
            "",
            gr.update(visible=True, value=items),
            gr.update(visible=True),
            gr.update(choices=delete_choices, value=None),
            ""
        )

    def on_delete(u, pid):
      #   Delete selected plant (by id), then reload UI state.

        username = _get_username(u)
        if not username:
            return load(u)

        if not pid:
            msg, empty_html, gal_upd, delrow_upd, dd_upd, _ = load(u)
            return msg, empty_html, gal_upd, delrow_upd, dd_upd, "⚠️ Please select a plant to delete."

        ok, msg_del = delete_plant(username, pid)
        msg, empty_html, gal_upd, delrow_upd, dd_upd, _ = load(u)
        return msg, empty_html, gal_upd, delrow_upd, dd_upd, ("✅ Deleted." if ok else f"❌ {msg_del}")

    refresh_btn.click(
        fn=load,
        inputs=[user_state],
        outputs=[info, empty_state, gallery, delete_row, plant_to_delete, del_status]
    )

    del_btn.click(
        fn=on_delete,
        inputs=[user_state, plant_to_delete],
        outputs=[info, empty_state, gallery, delete_row, plant_to_delete, del_status]

    )
    
    # Return wiring for auto-load on navigation
    return refresh_btn, load, [user_state], [info, empty_state, gallery, delete_row, plant_to_delete, del_status]



### Cell 16: ui/sensors_ui.py

In [ ]:
%%writefile ui/sensors_ui.py
import gradio as gr

from plants_manager import list_plants
from data_manager import get_latest_reading, get_sensor_history, sync_iot_data


def _get_username(user_state):
    return user_state.strip() if isinstance(user_state, str) else ""


def _plant_label(p: dict) -> str:
    pid = p.get("plant_id", "") or p.get("id", "")
    name = p.get("name") or ""
    species = p.get("species") or ""
    title = name or species or "Plant"
    return f"{title} ({pid})" if pid else title


def sensors_screen(user_state: gr.State):
    gr.Markdown("## 🌱 IoT Sensors")
    info = gr.Markdown()

    with gr.Row():
        plant_dd = gr.Dropdown(label="Choose a plant", choices=[], value=None, interactive=True)
        refresh_btn = gr.Button("🔄 Refresh", variant="secondary", scale=0)

    # Latest metrics (NO light)
    with gr.Row():
        m_temp = gr.HTML()
        m_hum = gr.HTML()
        m_soil = gr.HTML()

    history = gr.Dataframe(
        headers=["timestamp", "temp", "humidity", "soil"],
        datatype=["str", "number", "number", "number"],
        interactive=False,
        row_count=10,
        col_count=(4, "fixed"),
        label="Latest history (most recent first)",
    )

    def _metric_html(label: str, value):
        v = "N/A" if value is None else value
        return f"""
        <div class="metric">
          <div class="label">{label}</div>
          <div class="value">{v}</div>
        </div>
        """

    def load(u, chosen_pid):
        username = _get_username(u)

        if not username:
            return (
                "⚠️ Please login to view sensors data.",
                gr.update(choices=[], value=None),
                _metric_html("Temp (°C)", None),
                _metric_html("Humidity (%)", None),
                _metric_html("Soil", None),
                [],
            )

        plants = list_plants(username) or []
        choices = []
        for p in plants:
            pid = p.get("plant_id") or p.get("id")
            if pid:
                choices.append((_plant_label(p), pid))

        if not choices:
            return (
                f"🌱 No plants found. Add a plant in **Upload**, then come back.",
                gr.update(choices=[], value=None),
                _metric_html("Temp (°C)", None),
                _metric_html("Humidity (%)", None),
                _metric_html("Soil", None),
                [],
            )

        pid = chosen_pid or choices[0][1]

        # pull 1 fresh sample and store in Firestore
        sync_iot_data(pid)

        latest = get_latest_reading(pid)
        temp = latest.get("temp") if latest else None
        hum = latest.get("humidity") if latest else None
        soil = latest.get("soil") if latest else None

        hist = get_sensor_history(pid, limit=10) or []
        rows = []
        for r in hist:
            rows.append([
                r.get("timestamp"),
                r.get("temp"),
                r.get("humidity"),
                r.get("soil"),
            ])

        return (
            f"✅ Showing sensors for selected plant",
            gr.update(choices=choices, value=pid),
            _metric_html("Temp (°C)", temp),
            _metric_html("Humidity (%)", hum),
            _metric_html("Soil", soil),
            rows,
        )

    # Reactive: update when dropdown selection changes
    plant_dd.change(
        fn=load,
        inputs=[user_state, plant_dd],
        outputs=[info, plant_dd, m_temp, m_hum, m_soil, history],
    )

    # Manual refresh button (for syncing new IoT data)
    refresh_btn.click(
        fn=load,
        inputs=[user_state, plant_dd],
        outputs=[info, plant_dd, m_temp, m_hum, m_soil, history],
    )

    # Return components for external wiring (auto-load on navigation)
    return refresh_btn, load, [user_state, plant_dd], [info, plant_dd, m_temp, m_hum, m_soil, history]


### Cell 17: ui/dashboard_ui.py

In [ ]:
%%writefile ui/dashboard_ui.py

import html
import datetime as dt
import gradio as gr
import matplotlib.pyplot as plt

from plants_manager import list_plants
from data_manager import get_latest_reading, get_sensor_history, sync_iot_data


# =========================
# Helpers
# =========================

def _get_username(user_state):
    return user_state.strip() if isinstance(user_state, str) else ""

def _plant_label(p: dict) -> str:
    pid = p.get("plant_id", "") or p.get("id", "")
    name = p.get("name") or ""
    species = p.get("species") or ""
    title = name or species or "Plant"
    return f"{title} ({pid})" if pid else title


def _parse_ts(x):
    if x is None:
        return None
    if isinstance(x, dt.datetime):
        return x
    s = str(x).strip()
    for fmt in ("%Y-%m-%d %H:%M:%S", "%Y-%m-%dT%H:%M:%S", "%Y-%m-%d"):
        try:
            return dt.datetime.strptime(s[:19], fmt)
        except Exception:
            pass
    return None


# =========================
# Health logic
# =========================

def _health_eval(latest: dict):
    if not latest:
        return 0, "No data", ["No sensor data found for this plant yet."]

    temp = latest.get("temp")
    hum = latest.get("humidity")
    soil = latest.get("soil")

    issues = []
    score = 100

    if soil is not None:
        try:
            soil_v = float(soil)
            if soil_v < 30:
                issues.append("Soil is dry → consider watering")
                score -= 25
            elif soil_v > 70:
                issues.append("Soil is very wet → overwatering risk")
                score -= 20
        except Exception:
            pass

    if temp is not None:
        try:
            t = float(temp)
            if t < 15:
                issues.append("Temperature is low")
                score -= 15
            elif t > 30:
                issues.append("Temperature is high")
                score -= 15
        except Exception:
            pass

    if hum is not None:
        try:
            h = float(hum)
            if h < 35:
                issues.append("Humidity is low")
                score -= 10
            elif h > 75:
                issues.append("Humidity is high")
                score -= 10
        except Exception:
            pass

    score = max(0, min(100, score))

    if score >= 85:
        status = "Healthy"
    elif score >= 65:
        status = "Needs attention"
    else:
        status = "Unhealthy"

    if not issues:
        issues = ["Looks good based on the latest reading."]

    return score, status, issues


def _health_score_only(reading: dict) -> int:
    score, _, _ = _health_eval(reading)
    return score

# ======================================================
# Matplotlib  styling
# ======================================================
def _palette(is_dark: bool):
    """Color palette depending on current theme."""
    if is_dark:
        return {
            "bg": "#0b1220",
            "fg": "#e2e8f0",
            "grid": "#334155",
        }
    return {
        "bg": "#ffffff",
        "fg": "#0f172a",
        "grid": "#cbd5e1",
    }

def _styled_fig(is_dark: bool, size=(7, 3.2)):
    pal = _palette(is_dark)
    fig = plt.figure(figsize=size)
    fig.patch.set_facecolor(pal["bg"])
    ax = fig.add_subplot(111)

    ax.set_facecolor(pal["bg"])
    ax.tick_params(colors=pal["fg"])
    ax.xaxis.label.set_color(pal["fg"])
    ax.yaxis.label.set_color(pal["fg"])
    ax.title.set_color(pal["fg"])

    for spine in ["top", "right"]:
        ax.spines[spine].set_visible(False)
    for spine in ["left", "bottom"]:
        ax.spines[spine].set_color(pal["grid"])

    ax.grid(True, alpha=0.3, color=pal["grid"])
    return fig, ax

# def _style_axes(ax):
#     ax.set_facecolor("#0b1220")
#     for side in ["top", "right"]:
#         ax.spines[side].set_visible(False)
#     for side in ["left", "bottom"]:
#         ax.spines[side].set_color("#334155")

#     ax.tick_params(colors="#e2e8f0")
#     ax.yaxis.label.set_color("#e2e8f0")
#     ax.xaxis.label.set_color("#e2e8f0")
#     ax.title.set_color("#e2e8f0")
#     ax.grid(True, alpha=0.25)

# =========================
# Plot builders (Light-only / default matplotlib)
# =========================
def _line_plot(points, title, ylabel):
    fig = plt.figure(figsize=(7, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title(title, fontweight="bold")
    ax.set_xlabel("Time")
    ax.set_ylabel(ylabel)
    if points:
        xs, ys = zip(*points)
        ax.plot(xs, ys, linewidth=2.4)
    ax.grid(True, alpha=0.25)
    fig.autofmt_xdate()
    fig.tight_layout()
    return fig


def _hist_plot(values, title, xlabel):
    fig = plt.figure(figsize=(7, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title(title, fontweight="bold")
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Count")
    if values:
        ax.hist(values, bins=10, alpha=0.9)
    ax.grid(True, alpha=0.25)
    fig.tight_layout()
    return fig


def _scatter_plot(xs, ys, title, xlabel, ylabel):
    fig = plt.figure(figsize=(7, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title(title, fontweight="bold")
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if xs and ys:
        ax.scatter(xs, ys, s=45, alpha=0.85)
    ax.grid(True, alpha=0.25)
    fig.tight_layout()
    return fig


def _delta_plot(t, h, s):
    fig = plt.figure(figsize=(7, 3.2))
    ax = fig.add_subplot(111)

    ax.set_title("Change between samples (Δ)", fontweight="bold")
    ax.set_xlabel("Time")
    ax.set_ylabel("Δ value")

    def delta(points):
        return [(points[i][0], points[i][1] - points[i - 1][1]) for i in range(1, len(points))]

    if len(t) > 1:
        xs, ys = zip(*delta(t))
        ax.plot(xs, ys, label="Δ Temp")
    if len(h) > 1:
        xs, ys = zip(*delta(h))
        ax.plot(xs, ys, label="Δ Humidity")
    if len(s) > 1:
        xs, ys = zip(*delta(s))
        ax.plot(xs, ys, label="Δ Soil")

    ax.legend(frameon=False)
    ax.grid(True, alpha=0.25)
    fig.autofmt_xdate()
    fig.tight_layout()
    return fig

# =========================
# UI
# =========================

def dashboard_screen(user_state: gr.State):

    gr.Markdown("## 🌿 Plant Dashboard")
    gr.Markdown("Visual overview based on **real IoT data**.")

    info = gr.Markdown()

    with gr.Row():
        plant_dd = gr.Dropdown(label="Choose a plant", interactive=True)
        days_dd = gr.Dropdown(
            choices=[("Last 7 days", 7), ("Last 14 days", 14), ("Last 30 days", 30)],
            value=14,
            label="Range",
            interactive=True,
        )
        refresh_btn = gr.Button("Refresh", variant="secondary", scale=0)

    summary_html = gr.HTML()

    # -------- PLOTS (hidden by default) --------
    plots = gr.Column(visible=False)
    with plots:
        with gr.Row():
            p_soil_hist = gr.Plot()
            p_temp = gr.Plot()

        with gr.Row():
            p_hum = gr.Plot()
            p_soil = gr.Plot()

        with gr.Row():
            p_health = gr.Plot()
            p_scatter = gr.Plot()

    def load(u, pid, days):
        username = _get_username(u)

        if not username:
            return (
                "⚠️ Please login to view the dashboard.",
                gr.update(choices=[], value=None),
                "<b>Login required</b>",
                gr.update(visible=False),
                None, None, None, None, None, None
            )

        plants = list_plants(username) or []
        choices = [(_plant_label(p), p.get("plant_id") or p.get("id")) for p in plants if p.get("plant_id") or p.get("id")]

        if not choices:
            return (
                "No plants found.",
                gr.update(choices=[], value=None),
                "<b>No plants yet</b>",
                gr.update(visible=False),
                None, None, None, None, None, None
            )

        pid = pid or choices[0][1]
        sync_iot_data(pid)

        since = dt.datetime.utcnow() - dt.timedelta(days=int(days))
        hist = get_sensor_history(pid, limit=500) or []

        pts_t, pts_h, pts_s, pts_health = [], [], [], []
        xs_s, ys_h = [], []

        for r in hist:
            ts = _parse_ts(r.get("timestamp"))
            if not ts or ts < since:
                continue

            if r.get("temp") is not None:
                pts_t.append((ts, float(r["temp"])))
            if r.get("humidity") is not None:
                pts_h.append((ts, float(r["humidity"])))
            if r.get("soil") is not None:
                pts_s.append((ts, float(r["soil"])))

            pts_health.append((ts, _health_score_only(r)))

            if r.get("soil") is not None and r.get("humidity") is not None:
                xs_s.append(float(r["soil"]))
                ys_h.append(float(r["humidity"]))

        latest = get_latest_reading(pid)
        score, status, insights = _health_eval(latest)

        summary = f"<b>Status:</b> {status}<br><b>Score:</b> {score}<ul>"
        summary += "".join(f"<li>{html.escape(i)}</li>" for i in insights)
        summary += "</ul>"

        return (
            "Dashboard loaded.",
            gr.update(choices=choices, value=pid),
            summary,
            gr.update(visible=True),
            _hist_plot([v for _, v in pts_s], "Soil moisture distribution", "Soil"),
            _line_plot(pts_t, "Temperature (°C)", "°C"),
            _line_plot(pts_h, "Humidity (%)", "%"),
            _line_plot(pts_s, "Soil moisture trend", "Soil"),
            _delta_plot(pts_t, pts_h, pts_s),
            _scatter_plot(xs_s, ys_h, "Soil vs Humidity", "Soil", "Humidity")
        )


    for c in (plant_dd, days_dd):
        c.change(
            load,
            inputs=[user_state, plant_dd, days_dd],
            outputs=[info, plant_dd, summary_html, plots,p_soil_hist, p_temp, p_hum, p_soil,
            p_health, p_scatter],
        )
    # # Reactive: update when plant dropdown selection changes
    # plant_dd.change(
    #     load,
    #     inputs=[user_state, plant_dd, days_dd],
    #     outputs=[
    #         info, plant_dd, summary_html,
    #         plots_wrap,
    #         p_soil_hist, p_temp, p_hum, p_soil,
    #         p_health, p_scatter
    #     ],
    # )

    # # Reactive: update when days range changes
    # days_dd.change(
    #     load,
    #     inputs=[user_state, plant_dd, days_dd],
    #     outputs=[
    #         info, plant_dd, summary_html,
    #         plots_wrap,
    #         p_soil_hist, p_temp, p_hum, p_soil,
    #         p_health, p_scatter
    #     ],
    # )

    # Manual refresh button
    refresh_btn.click(
        load,
        inputs=[user_state, plant_dd, days_dd],
        outputs=[
            info, plant_dd, summary_html,
            plots,
            p_soil_hist, p_temp, p_hum, p_soil,
            p_health, p_scatter
        ],
    )

    # Return components for external wiring (auto-load on navigation)
    return refresh_btn, load, [user_state, plant_dd, days_dd], [
        info, plant_dd, summary_html,
        plots,
        p_soil_hist, p_temp, p_hum, p_soil,
        p_health, p_scatter
    ]


### Cell 18: ui/search_ui.py

In [ ]:
%%writefile ui/search_ui.py
import gradio as gr
from data_manager import PlantRAG

_RAG = None

def _get_rag() -> PlantRAG:
    global _RAG
    if _RAG is None:
        _RAG = PlantRAG()
    return _RAG


def _fmt_paper_lines(chunks, limit: int = 3) -> str:
    chunks = (chunks or [])[:limit]
    if not chunks:
        return "_No sources available._"

    lines = []
    for i, c in enumerate(chunks, start=1):
        title = (c.get("title") or "Untitled").strip()
        url = (c.get("url") or "").strip()

        meta = c.get("metadata") or {}
        authors = (meta.get("authors") or "").strip()
        journal = (meta.get("journal") or "").strip()
        year = (meta.get("year") or "").strip()
        doi = (meta.get("doi") or "").strip()

        # --- choose best link: url if exists, else DOI link ---
        link_url = url
        if not link_url and doi:
            link_url = f"https://doi.org/{doi}"

        # Title line (clickable if we have either url or doi)
        if link_url:
            lines.append(f"**{i}. [{title}]({link_url})**")
        else:
            lines.append(f"**{i}. {title}**")

        # (optional) clean authors a bit
        if authors:
            authors = authors.split("E-mail")[0].split("Accepted")[0].strip()
            lines.append(f"- 👤 Authors: {authors}")

        # Metadata lines (only if exist) - remove "Unknown journal"
        if journal:
            if year:
                lines.append(f"- 📰 {journal} ({year})")
            else:
                lines.append(f"- 📰 {journal}")
        elif year:
            lines.append(f"- 🗓️ Year: {year}")

        # DOI as a clickable link (not just code)
        if doi:
            lines.append(f"- 🔗 DOI: [{doi}](https://doi.org/{doi})")

        lines.append("")

    return "\n".join(lines).strip()



def run_query(question: str, top_k: int = 3) -> str:
    q = (question or "").strip()
    if not q:
        return "⚠️ Please enter a question."

    out = _get_rag().query(q, top_k=int(top_k))

    answer = (out.get("response") or "").strip()
    chunks = out.get("chunks") or []
    papers_found = int(out.get("papers_found") or len(chunks))

    md = []
    md.append("### Research-based Answer")
    md.append("")
    md.append(answer if answer else "_No answer returned._")
    md.append("")
    md.append("---")
    md.append(f"**Found {papers_found} relevant papers:**")
    md.append("")
    md.append(_fmt_paper_lines(chunks, limit=min(int(top_k), 5)))

    return "\n".join(md)


def search_screen():
    gr.Markdown("## Search Articles")
    gr.Markdown("Ask a question. We'll search the knowledge base and return the most relevant papers.")

    question = gr.Textbox(
        label="Ask your ecological question",
        placeholder="e.g., how do I know how much water my plant needs?",
        lines=2,
    )

    with gr.Row():
        top_k = gr.Slider(1, 5, value=3, step=1, label="Number of papers to search")
        submit = gr.Button("Submit", variant="primary")
        clear = gr.Button("Clear")

    output = gr.Markdown(value="")

    submit.click(run_query, inputs=[question, top_k], outputs=[output])
    question.submit(run_query, inputs=[question, top_k], outputs=[output])
    clear.click(lambda: ("", 3, ""), outputs=[question, top_k, output])

    return output


### Cell 19: ui/upload_ui.py

In [ ]:
%%writefile ui/upload_ui.py
import gradio as gr
from plants_manager import add_plant_with_image


def _get_username(user_state):
    """Extract username string from Gradio State (or empty string)."""
    return user_state.strip() if isinstance(user_state, str) else ""


def upload_screen(user_state: gr.State):
    """Upload UI: collects image + metadata and delegates saving to plants_manager."""
    gr.Markdown("## 📷 Upload Plant Image")

    with gr.Row():

        with gr.Column(scale=6):
            image_in = gr.Image(label="Upload a plant photo", type="pil")


        with gr.Column(scale=6):
            plant_name = gr.Textbox(label="Plant name", placeholder="e.g., My Basil")
            species = gr.Textbox(label="Species (optional)", placeholder="e.g., Basil / Monstera / Cactus")
            save_btn = gr.Button("Save to My Plants", variant="primary")
            status = gr.Markdown("")

    def on_save(u, img, name, sp):
        username = _get_username(u)

        if not username:
            return "⚠️ Please login first.", gr.update(), gr.update(), gr.update()
        if img is None:
            return "⚠️ Please upload an image.", gr.update(), gr.update(), gr.update()
        if not str(name).strip():
            return "⚠️ Please enter plant name.", gr.update(), gr.update(), gr.update()

        ok, plant_id_or_err = add_plant_with_image(
            username=username,
            name=name,
            species=sp or "",
            pil_image=img,
        )

        if not ok:
            return f"❌ {plant_id_or_err}", gr.update(), gr.update(), gr.update()

        # Success: clear the form for next upload
        return f"✅ Saved plant **{name}** (id: `{plant_id_or_err}`)", None, "", ""

    save_btn.click(
        fn=on_save,
        inputs=[user_state, image_in, plant_name, species],
        outputs=[status, image_in, plant_name, species],
    )


---
## 🚀 Final Cell: Launch Application

In [ ]:
# Run the application with public sharing enabled
!python app.py